<a href="https://colab.research.google.com/github/MithunKiniStar/AutoEncodersFaceSketch/blob/main/Deep_Convolutional_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Deep Convolutional autoencoder**

Data Preprocessing

In [1]:
import numpy as np
import tensorflow as tf
import keras 
from keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Dropout, Input
from keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm 
import os
import re 
# to get the files in proper order
def sorted_alphanumeric(data):  
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)


# defining the size of image 
SIZE = 256

image_path = '/content/photos'
img_array = []

sketch_path = '/content/sketches'
sketch_array = []

image_file = sorted_alphanumeric(os.listdir(image_path))
sketch_file = sorted_alphanumeric(os.listdir(sketch_path))


for i in tqdm(image_file):
    image = cv2.imread(image_path + '/' + i,1)
    print("Image is ",image)
    
    # as opencv load image in bgr format converting it to rgb
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # resizing images 
    image = cv2.resize(image, (SIZE, SIZE))
    
    # normalizing image 
    image = image.astype('float32') / 255.0
    
    #appending normal normal image    
    img_array.append(img_to_array(image))
    # Image Augmentation
    
    # horizontal flip 
    img1 = cv2.flip(image,1)
    img_array.append(img_to_array(img1))
     #vertical flip 
    img2 = cv2.flip(image,-1)
    img_array.append(img_to_array(img2))
     #vertical flip 
    img3 = cv2.flip(image,-1)
    # horizontal flip
    img3 = cv2.flip(img3,1)
    img_array.append(img_to_array(img3))
    # rotate clockwise 
    img4 = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    img_array.append(img_to_array(img4))
    # flip rotated image 
    img5 = cv2.flip(img4,1)
    img_array.append(img_to_array(img5))
     # rotate anti clockwise 
    img6 = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    img_array.append(img_to_array(img6))
    # flip rotated image 
    img7 = cv2.flip(img6,1)
    img_array.append(img_to_array(img7))
  
    
for i in tqdm(sketch_file):
    image = cv2.imread(sketch_path + '/' + i,1)
    
    # as opencv load image in bgr format converting it to rgb
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # resizing images 
    image = cv2.resize(image, (SIZE, SIZE))
    
    # normalizing image 
    image = image.astype('float32') / 255.0
    # appending normal sketch image
    sketch_array.append(img_to_array(image))
    
    #Image Augmentation
    # horizontal flip 
    img1 = cv2.flip(image,1)
    sketch_array.append(img_to_array(img1))
     #vertical flip 
    img2 = cv2.flip(image,-1)
    sketch_array.append(img_to_array(img2))
     #vertical flip 
    img3 = cv2.flip(image,-1)
    # horizontal flip
    img3 = cv2.flip(img3,1)
    sketch_array.append(img_to_array(img3))
    # rotate clockwise 
    img4 = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    sketch_array.append(img_to_array(img4))
    # flip rotated image 
    img5 = cv2.flip(img4,1)
    sketch_array.append(img_to_array(img5))
     # rotate anti clockwise 
    img6 = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    sketch_array.append(img_to_array(img6))
    # flip rotated image 
    img7 = cv2.flip(img6,1)
    sketch_array.append(img_to_array(img7))

  7%|▋         | 13/188 [00:00<00:02, 63.92it/s]

Image is  [[[202 175 101]
  [202 175 101]
  [202 175 101]
  ...
  [199 173 106]
  [199 173 106]
  [199 173 106]]

 [[202 175 101]
  [202 175 101]
  [202 175 101]
  ...
  [199 173 106]
  [199 173 106]
  [199 173 106]]

 [[202 175 101]
  [202 175 101]
  [202 175 101]
  ...
  [199 173 106]
  [199 173 106]
  [199 173 106]]

 ...

 [[ 13  18  19]
  [ 23  28  29]
  [ 26  31  32]
  ...
  [ 27  56  83]
  [ 27  56  83]
  [ 29  58  85]]

 [[ 27  35  35]
  [ 24  32  32]
  [ 18  26  26]
  ...
  [ 29  59  86]
  [ 21  50  77]
  [ 30  59  86]]

 [[ 27  35  35]
  [ 24  32  32]
  [ 18  26  26]
  ...
  [ 28  58  85]
  [ 21  50  77]
  [ 32  61  88]]]
Image is  [[[207 180 106]
  [207 180 106]
  [207 180 106]
  ...
  [214 186 115]
  [214 187 113]
  [214 187 113]]

 [[207 180 106]
  [207 180 106]
  [207 180 106]
  ...
  [215 187 116]
  [215 188 114]
  [215 188 114]]

 [[207 180 106]
  [207 180 106]
  [207 180 106]
  ...
  [216 188 117]
  [216 189 115]
  [216 189 115]]

 ...

 [[ 90 222 246]
  [ 88 224 250]


 15%|█▍        | 28/188 [00:00<00:02, 68.58it/s]

Image is  [[[156 123  43]
  [157 124  44]
  [157 124  44]
  ...
  [159 128  53]
  [159 128  53]
  [159 128  53]]

 [[157 124  44]
  [158 125  45]
  [158 125  45]
  ...
  [159 128  53]
  [159 128  53]
  [159 128  53]]

 [[159 126  46]
  [159 126  46]
  [159 126  46]
  ...
  [159 128  53]
  [159 128  53]
  [159 128  53]]

 ...

 [[171 135  59]
  [168 133  59]
  [165 133  62]
  ...
  [105 102  97]
  [111 106  97]
  [113 109  98]]

 [[177 138  64]
  [171 133  61]
  [164 131  62]
  ...
  [ 89  97 104]
  [ 99 104 107]
  [101 106 107]]

 [[176 137  63]
  [171 133  61]
  [164 130  64]
  ...
  [ 79  90 104]
  [ 88  98 108]
  [ 92 101 110]]]
Image is  [[[255 221 114]
  [255 221 114]
  [255 220 116]
  ...
  [255 217 125]
  [255 217 125]
  [255 217 125]]

 [[255 221 114]
  [255 221 114]
  [255 220 116]
  ...
  [255 217 125]
  [255 217 125]
  [255 217 125]]

 [[255 221 114]
  [255 221 114]
  [255 220 116]
  ...
  [255 217 125]
  [255 217 125]
  [255 217 125]]

 ...

 [[206 164  97]
  [195 157  93]


 23%|██▎       | 44/188 [00:00<00:01, 72.45it/s]

...

 91%|█████████▏| 172/188 [00:02<00:00, 74.97it/s]

Image is  [[[234 207 126]
  [234 207 126]
  [234 207 126]
  ...
  [233 208 134]
  [233 208 134]
  [233 208 134]]

 [[234 207 126]
  [234 207 126]
  [235 208 127]
  ...
  [233 208 134]
  [233 208 134]
  [233 208 134]]

 [[234 207 126]
  [235 208 127]
  [235 208 127]
  ...
  [233 208 134]
  [233 208 134]
  [233 208 134]]

 ...

 [[ 40  44  45]
  [ 40  44  45]
  [ 39  43  44]
  ...
  [  0   1   0]
  [  0   1   1]
  [  0   1   0]]

 [[ 43  47  48]
  [ 36  40  41]
  [ 29  33  34]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[ 43  47  48]
  [ 36  40  41]
  [ 29  33  34]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]]
Image is  [[[246 214 131]
  [246 214 131]
  [246 214 131]
  ...
  [240 211 136]
  [240 211 136]
  [240 211 136]]

 [[246 214 131]
  [246 214 131]
  [246 214 131]
  ...
  [240 211 136]
  [240 211 136]
  [240 211 136]]

 [[246 214 131]
  [246 214 131]
  [246 214 131]
  ...
  [240 211 136]
  [240 211 136]
  [240 211 136]]

 ...

 [[  0   0   0]
  [  0   0   0]


100%|██████████| 188/188 [00:02<00:00, 72.27it/s]

Image is  [[[  0   0   9]
  [  0   0  12]
  [  2   2  14]
  ...
  [231 204 124]
  [231 204 124]
  [230 203 123]]

 [[  0   0   6]
  [  0   0   4]
  [  0   0   4]
  ...
  [232 205 125]
  [231 204 124]
  [230 203 123]]

 [[ 20  15  14]
  [ 13   8   5]
  [  7   2   0]
  ...
  [232 205 125]
  [231 204 124]
  [231 204 124]]

 ...

 [[221 192 117]
  [221 192 117]
  [221 192 117]
  ...
  [ 25  33  40]
  [ 26  35  39]
  [ 27  36  40]]

 [[221 192 117]
  [221 192 117]
  [220 191 116]
  ...
  [ 26  34  41]
  [ 25  33  40]
  [ 25  33  40]]

 [[221 192 117]
  [221 192 117]
  [220 191 116]
  ...
  [ 26  34  41]
  [ 25  33  40]
  [ 25  33  40]]]
Image is  [[[245 215 132]
  [244 214 131]
  [243 213 130]
  ...
  [238 211 131]
  [238 211 131]
  [238 211 131]]

 [[245 215 132]
  [244 214 131]
  [243 213 130]
  ...
  [238 211 131]
  [238 211 131]
  [238 211 131]]

 [[245 215 132]
  [245 215 132]
  [244 214 131]
  ...
  [239 212 132]
  [239 212 132]
  [239 212 132]]

 ...

 [[229 192 108]
  [230 193 109]



100%|██████████| 188/188 [00:02<00:00, 88.48it/s]


In [2]:
print("Total number of sketch images:",len(sketch_array))
print("Total number of images:",len(img_array))

Total number of sketch images: 1504
Total number of images: 1504


In [3]:
print("Total number of sketch images:",len(sketch_array))
print("Total number of images:",len(img_array))
train_sketch_image = sketch_array[:1400]
train_image = img_array[:1400]
test_sketch_image = sketch_array[1400:]
test_image = img_array[1400:]
# reshaping
train_sketch_image = np.reshape(train_sketch_image,(len(train_sketch_image),SIZE,SIZE,3))
train_image = np.reshape(train_image, (len(train_image),SIZE,SIZE,3))
print('Train color image shape:',train_image.shape)
test_sketch_image = np.reshape(test_sketch_image,(len(test_sketch_image),SIZE,SIZE,3))
test_image = np.reshape(test_image, (len(test_image),SIZE,SIZE,3))
print('Test color image shape',test_image.shape)

Total number of sketch images: 1504
Total number of images: 1504
Train color image shape: (1400, 256, 256, 3)
Test color image shape (104, 256, 256, 3)


Model for Face to Sketch

In [4]:
import keras
from keras import layers

input_img = keras.Input(shape=(256, 256, 3))


x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu')(x)
x = layers.UpSampling2D((2, 2))(x)
#decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
x = tf.keras.layers.Conv2DTranspose(8,(3,3), padding = 'valid')(x)
decoder_output = tf.keras.layers.Conv2DTranspose(3,(3,3), padding = 'valid')(x)


autoencoder = keras.Model(input_img, decoder_output)
#autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = 'mean_absolute_error',
              metrics = ['acc'])
decoder_output.shape

TensorShape([None, 256, 256, 3])

In [ ]:
# Training for photo to sketch
from keras.callbacks import TensorBoard
# Train the autoencoder model
train_image.shape
#hist=autoencoder.fit(train_image, train_sketch_image, epochs = 5, verbose = 0)

hist=autoencoder.fit(train_image, train_sketch_image,
                epochs=100,
                batch_size=128,
                shuffle=True,
                validation_data=(test_image, test_sketch_image))



plt.plot(hist.history["loss"]);
plt.xlabel('Epochs');
plt.ylabel('Training Error');

Epoch 1/100
 4/11 [=========>....................] - ETA: 1:44 - loss: 0.7518 - acc: 0.0214

In [ ]:
decoded_imgs = autoencoder.predict(test_image)

n = 10
plt.figure(figsize=(20, 4))
for i in range(1, n + 1):
    # Display original
    ax = plt.subplot(2, n, i)
    plt.imshow(test_image[i].reshape(256, 256, 3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.imshow(decoded_imgs[i].reshape(256, 256, 3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()